In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path = "/content/drive/My Drive/Bathiya/plant-seedlings-classification/train"
test_path = "/content/drive/My Drive/Bathiya/plant-seedlings-classification/test"

In [ ]:
import os
import os,glob
import cv2
import numpy as np

In [ ]:
from sklearn import preprocessing as skLearnPreprocess
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
def getPath (path):
  return glob.glob(path)

paths = getPath(train_path +"/*")

##  Read Images

In [ ]:
images = []
classes=[]
missing=0
for class_folder_name in os.listdir(train_path):
    class_folder_path = os.path.join(train_path, class_folder_name)
    class_label = class_folder_name
    
    for image_path in glob.glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image_bgr is None: # if the file contain any missing value ignore it
          missing += 1
          continue
        images.append(image_bgr)
        
        classes.append(class_label)

## Resize images

In [ ]:
# resize image
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(224,224), interpolation = cv2.INTER_CUBIC)
  return res

#save resized images into images.
images = [resize_images(img) for img in images]

In [ ]:
classes = np.array(classes).reshape(-1,1)

In [ ]:
images = np.array(images)

## Label Encoding

In [ ]:
def encodeLabels(y_train, y_valid):
  le = skLearnPreprocess.LabelEncoder()
  y_train = le.fit_transform(y_train)
  y_valid = le.fit_transform(y_valid)
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=12)
  y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=12)
  y_valid = np.array(y_valid)
  y_train = np.array(y_train)

  return y_train , y_valid

## Build Model

In [ ]:
def buildmodel():
    data_augmentation = keras.Sequential(
        [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
    )

    base_model = keras.applications.MobileNetV2(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(224, 224, 3),
        include_top=False,
    )  # Do not include the ImageNet classifier at the top.

    # Freeze the base_model
    base_model.trainable = False
    inputs = keras.Input(shape=(224, 224, 3))
    x = data_augmentation(inputs)  # Apply random data augmentation

    # Pre-trained Xception weights requires that input be scaled
    # from (0, 255) to a range of (-1., +1.), the rescaling layer
    # outputs: `(inputs * scale) + offset`
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(x)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(12,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    
    model.compile(
    optimizer= tf.keras.optimizers.Adam(epsilon=0.0001),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
    )
    
    return(model)

## Cross Validation

In [ ]:
kFold = StratifiedKFold(n_splits=5) 
accuracies = []
losses = []

In [ ]:
def train_evaluate(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 10)
    return model.evaluate(x_test, y_test)

In [ ]:
idx = 0
for train, test in kFold.split(images, classes):
  print("Starting Fold", idx)
  model = buildmodel()
  print(classes[train])
  y_train, y_valid = encodeLabels(classes[train],classes[test])

  print(y_train)

  scores = train_evaluate(model, images[train], y_train, images[test],y_valid)
  accuracies.append(scores[1])
  losses.append (scores[0])
  print ("Validation accuracy: ", scores[1])
  print ("Validation loss", scores[0])

  idx += 1

In [ ]:
print(accuracies)
print(np.mean(accuracies))
print(np.mean(losses))

[0.8389473557472229, 0.8399999737739563, 0.8505263328552246, 0.8463158011436462, 0.8642105460166931]
0.8480000019073486
0.44977362751960753
